<a href="https://colab.research.google.com/github/as2c/MLP---Cosmology/blob/main/Data/density_contrast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
!git clone https://github.com/jveitchmichaelis/pygadgetreader

fatal: destination path 'pygadgetreader' already exists and is not an empty directory.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/pygadgetreader
!python setup.py build
!python setup.py install

/content/pygadgetreader
running build
running build_py
running install
running bdist_egg
running egg_info
writing pyGadgetReader.egg-info/PKG-INFO
writing dependency_links to pyGadgetReader.egg-info/dependency_links.txt
writing requirements to pyGadgetReader.egg-info/requires.txt
writing top-level names to pyGadgetReader.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'pyGadgetReader.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pygadgetreader
copying build/lib/pygadgetreader/__init__.py -> build/bdist.linux-x86_64/egg/pygadgetreader
creating build/bdist.linux-x86_64/egg/readgadget
copying build/lib/readgadget/readpstar.py -> build/bdist.linux-x86_64/egg/readgadget
copying build/lib/readgadget/readrockstar.py -> build/bdist.linux-x86_64/egg/readgadget
copying build/lib/readgadget/readgadget.py -> build/bdist.linux-x8

In [ ]:
from pygadgetreader import *

In [ ]:
vel = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','vel','dm')
vx = vel[:,0].tolist()
vy = vel[:,1].tolist()
vz = vel[:,2].tolist()
mass = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','mass','dm').tolist()
particle_id = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','pid','dm').tolist()
position = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','pos','dm')
x = position[:,0].tolist()
y = position[:,1].tolist()
z = position[:,2].tolist()
data = {'particle_id': particle_id,
        'x': x,
        'y': y,
        'z': z,
        'vx': vx,
        'vy': vy,
        'vz': vz,
        'mass': mass}
df = pd.DataFrame(data)

Returning DM    Velocities
Returning DM    Mass
Returning DM    Particle IDs
Returning DM    Positions


In [ ]:
df.head()

,particle_id,x,y,z,vx,vy,vz,mass
0,0,192.691193,231.858017,225.984268,-659.554321,-419.040802,-489.896637,2.02478
1,7,809.101929,210.431351,214.217163,-702.608582,-530.673584,-534.257935,2.02478
2,56,1437.270874,198.266098,201.348267,-689.156860,-608.542847,-615.787476,2.02478
3,63,2043.859375,196.952347,187.470413,-797.107727,-646.467102,-686.565735,2.02478
4,64,2664.605225,204.039200,170.532455,-818.485474,-626.312866,-768.481689,2.02478


In [ ]:
df = df.sort_values( by =['x'])
df.drop(['mass'], axis = 1, inplace = True)

In [ ]:
df.insert(1, 'in-halo', 0)

The following line of code has to be changed, since we only assign a random value of 0 or 1 for each particle's in-halo value.

In [ ]:
df['in-halo'] = np.random.choice([0, 1], df.shape[0])
# df.head(10)

In [ ]:
data = df.copy(deep = True)
data.x = data.x / 1000
data.y = df.y / 1000
data.z = df.z / 1000
print(data.x.max())
print(data.y.max())
print(data.z.max())

159.846765625
159.85021875
159.824359375


In [ ]:
data.head()

,particle_id,in-halo,x,y,z,vx,vy,vz
0,0,0,0.192691,0.231858,0.225984,-659.554321,-419.040802,-489.896637
1,7,0,0.809102,0.210431,0.214217,-702.608582,-530.673584,-534.257935
2,56,0,1.437271,0.198266,0.201348,-689.156860,-608.542847,-615.787476
3,63,0,2.043859,0.196952,0.187470,-797.107727,-646.467102,-686.565735
4,64,1,2.664605,0.204039,0.170532,-818.485474,-626.312866,-768.481689


In [ ]:
box_size = 159
boundary = 20

d = data[data.x > boundary]

d = d[data.x < box_size - boundary]

d = d[data.y > boundary]

d = d[data.y < box_size - boundary]

d = d[data.z > boundary]

d = d[data.z < box_size - boundary]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
from tqdm import tqdm
def box_mass(r_cutoff, df, d):
    m = len(d)
    mass = pd.DataFrame(np.zeros(m),columns=['den_contrast'])
    for i in tqdm(range(m)):
        wx=d.iloc[i]['x']
        wy=d.iloc[i]['y']
        wz=d.iloc[i]['z']
        ix = np.searchsorted(df[:,2] , wx-r_cutoff)
        jx = np.searchsorted(df[:,2] , wx+r_cutoff)

        temp = np.where((df[ix:jx,3] < wy+r_cutoff) & (df[ix:jx,3] > wy-r_cutoff) & (df[ix:jx,4] < wz+r_cutoff) & (df[ix:jx,4] > wz-r_cutoff))
    
        mass.den_contrast[i] = len(temp[0])
    return mass

In [ ]:
n = 100000

sample = d.sample(n, random_state = 1)
sample_density = sample[['particle_id', 'in-halo']]
sample_density.reset_index(drop = True, inplace = True)
# sample_density.head()

In [ ]:
print(data)

          particle_id  in-halo           x  ...          vx          vy          vz
0                   0        0    0.192691  ... -659.554321 -419.040802 -489.896637
1                   7        0    0.809102  ... -702.608582 -530.673584 -534.257935
2                  56        0    1.437271  ... -689.156860 -608.542847 -615.787476
3                  63        0    2.043859  ... -797.107727 -646.467102 -686.565735
4                  64        1    2.664605  ... -818.485474 -626.312866 -768.481689
...               ...      ...         ...  ...         ...         ...         ...
16777211     16777133        1  157.062375  ... -701.834167 -397.481934 -357.832153
16777212     16777170        0  157.688234  ... -678.720825 -381.776428 -408.717957
16777213     16777173        0  158.317078  ... -681.724609 -368.299011 -465.415466
16777214     16777194        0  158.937750  ... -697.889648 -382.313843 -450.085297
16777215     16777197        1  159.574641  ... -637.320312 -410.518890 -407

In [ ]:
data = data.to_numpy()

In [ ]:
len(df)

16777216

In [ ]:
rc = np.array([])
for i in range(20, 23, 1):
  r =  0.6 * i + 0.4
  rc = np.append(rc, r)
  density = box_mass(r , data, sample)
  density = density.div((2*r)**3)
  sample_density = pd.concat([sample_density, density], axis = 1)
  sample_density = sample_density.rename(columns={'den_contrast': 'den_contrast for {}'.format(r)})
sample_density.head()

100%|██████████| 100000/100000 [24:33<00:00, 67.87it/s]


,particle_id,in-halo,den_contrast for 12.4,den_contrast for 13.0,den_contrast for 13.6
0,6734555,0,3.889729,3.921313,3.950924
1,12718733,1,3.913921,3.926491,3.951023
2,6470459,1,3.949455,3.967968,3.985460
3,12098249,1,3.909594,3.923020,3.950924
4,4799280,1,3.946111,3.939520,3.954750


In [ ]:
sample_density.to_csv('/content/drive/MyDrive/sample_density_hunthou.csv', index=False)

In [ ]:
rho_bar = len(df) / (159.8**3)
sample_density.iloc[:,2:] -= rho_bar
sample_density.iloc[:,2:] /= rho_bar
print(sample_density)

       particle_id  in-halo  ...  den_contrast for 13.0  den_contrast for 13.6
0          6734555        0  ...              -0.046234              -0.039032
1         12718733        1  ...              -0.044974              -0.039007
2          6470459        1  ...              -0.034886              -0.030631
3         12098249        1  ...              -0.045819              -0.039032
4          4799280        1  ...              -0.041805              -0.038101
...            ...      ...  ...                    ...                    ...
99995     13941529        0  ...              -0.046026              -0.039032
99996     10307225        1  ...              -0.045736              -0.039032
99997      5916846        0  ...              -0.046206              -0.039032
99998     13686590        0  ...              -0.044061              -0.038705
99999      9026520        0  ...              -0.045583              -0.039032

[100000 rows x 5 columns]


In [ ]:
sample_density.to_csv('/content/drive/MyDrive/density_contrast_hunthou.csv', index=False)

The above gives the physical properties that we were looking for, namely the density contrasts for each particle encompassed by boxes of different sizes.